In [ ]:
import os
import scipy.io as sio

AD = sio.loadmat("/kaggle/input/eeg-dataset/AD.mat")
MCI = sio.loadmat("/kaggle/input/eeg-dataset/MCI.mat")
Normal = sio.loadmat("/kaggle/input/eeg-dataset/normal.mat")

In [ ]:
AD

In [ ]:
import numpy as np
AD_Data = []
for i in range(len(AD['AD'][0])):
  a = AD['AD'][0][i][0]
  data = np.transpose(a, (2, 0, 1))
  for j in range(data.shape[0]):
    AD_Data.append(data[j])

AD_Data = np.array(AD_Data)
AD_Data.shape

In [ ]:
AD_Data

In [ ]:
MCI_Data = []
for i in range(len(MCI['MCI'][0])):
  a = MCI['MCI'][0][i][0]
  data = np.transpose(a, (2, 0, 1))
  for j in range(data.shape[0]):
    MCI_Data.append(data[j])

MCI_Data = np.array(MCI_Data)
MCI_Data.shape

In [ ]:
Normal_Data = []
for i in range(len(Normal['normal'][0])):
  a = Normal['normal'][0][i][0]
  data = np.transpose(a, (2, 0, 1))
  for j in range(data.shape[0]):
    Normal_Data.append(data[j])

Normal_Data = np.array(Normal_Data)
Normal_Data.shape

In [ ]:
Data = []
label = []
for i in range(Normal_Data.shape[0]):
  Data.append(Normal_Data[i])
  label.append(0)

for j in range(MCI_Data.shape[0]):
  Data.append(MCI_Data[j])
  label.append(1)

for k in range(AD_Data.shape[0]):
  Data.append(AD_Data[k])
  label.append(2)

In [ ]:
Data = np.array(Data)
label = np.array(label)
print(Data.shape)
print(label.shape)

In [ ]:
Ndata=[]
for i in  range(len(Data)):
    std=np.std(Data[i],axis=1)
    mean=np.mean(Data[i],axis=1)
    Data[i]=(Data[i].transpose()-mean.transpose()).transpose()
    Data[i]=(Data[i].transpose()/std.transpose()).transpose()
    Ndata.append(Data[i])

In [ ]:
Ndata

In [ ]:
from matplotlib import pyplot as plt
plt.plot(Ndata[0][0])

In [ ]:
from keras.utils import to_categorical
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

num_classes = 3
Y = to_categorical(label, num_classes)
x,y = shuffle(Data, Y, random_state = 2)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 2)
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

In [ ]:
# Reshape the input data
X_train = X_train.reshape((3489, 600, 4))

In [ ]:
X_train.shape

In [ ]:
y_train.shape

# Data Augmentation

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, BatchNormalization, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def sliding_window_augmentation(X, y, window_size, step_size):
    n_samples, n_timesteps, n_features = X.shape
    augmented_X = []
    augmented_y = []

    for sample, label in zip(X, y):
        for start in range(0, n_timesteps - window_size + 1, step_size):
            window = sample[start:start+window_size]
            augmented_X.append(window)
            augmented_y.append(label)

    return np.array(augmented_X), np.array(augmented_y)

# Perform data augmentation
window_size = int(600 * 0.75)  # 75% of original size w
step_size = 4  # distance of sliding window n

X_augmented, y_augmented = sliding_window_augmentation(X_train, y_train, window_size, step_size)

print(f"Augmented X shape: {X_augmented.shape}")
print(f"Augmented y shape: {y_augmented.shape}")

In [ ]:
y_augmented

# **Model with 5-Fold Cross Validation**

In [ ]:
# Building the model layers
def create_model(input_shape, num_classes):
    model = Sequential([
        Conv1D(24, kernel_size=3, activation='leaky_relu', input_shape=input_shape),
        BatchNormalization(),
        Conv1D(16, kernel_size=3, activation='leaky_relu'),
        BatchNormalization(),
        Conv1D(8, kernel_size=3, activation='leaky_relu'),
        BatchNormalization(),
        Conv1D(4, kernel_size=3, activation='leaky_relu'),
        BatchNormalization(),
        Flatten(),
        Dense(64, activation='leaky_relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    optimizer = Adam(learning_rate=0.00002)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Perform 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 1
confusion_matrices = []

for train_index, test_index in kf.split(X_augmented):
    print(f"Fold {fold}")

    X_train_fold, X_test_fold = X_augmented[train_index], X_augmented[test_index]
    y_train_fold, y_test_fold = y_augmented[train_index], y_augmented[test_index]

    model = create_model(input_shape=(X_train_fold.shape[1], X_train_fold.shape[2]), num_classes=3)

    # Train the model
    model.fit(X_train_fold, y_train_fold, epochs=20, batch_size=16, validation_split=0.2, verbose=1)

    # Evaluate the model
    y_pred = model.predict(X_test_fold)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test_fold, axis=1)

    # Create confusion matrix
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    confusion_matrices.append(cm)

    print(f"Confusion Matrix for Fold {fold}:")
    print(cm)
    print("\n")

    fold += 1

# Calculate and print the average confusion matrix
avg_cm = np.mean(confusion_matrices, axis=0)
print("Average Confusion Matrix:")
print(avg_cm)

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Initialize lists to store ROC curve data and AUC values
fpr_list = []
tpr_list = []
roc_auc_list = []

# Perform 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 1

for train_index, test_index in kf.split(X_augmented):
    print(f"Fold {fold}")

    X_train_fold, X_test_fold = X_augmented[train_index], X_augmented[test_index]
    y_train_fold, y_test_fold = y_augmented[train_index], y_augmented[test_index]

    model = create_model(input_shape=(X_train_fold.shape[1], X_train_fold.shape[2]), num_classes=3)

    # Train the model
    model.fit(X_train_fold, y_train_fold, epochs=20, batch_size=16, validation_split=0.2, verbose=1)

    # Evaluate the model
    y_pred = model.predict(X_test_fold)

    # Compute ROC curve and AUC for each class
    n_classes = y_test_fold.shape[1]
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_fold[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Store the ROC curve data and AUC values
    fpr_list.append(fpr)
    tpr_list.append(tpr)
    roc_auc_list.append(roc_auc)

    # Plot ROC curves for this fold
    plt.figure()
    colors = ['blue', 'red', 'green']
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for Fold {fold}')
    plt.legend(loc="lower right")
    plt.show()

    fold += 1

# Calculate and print the average AUC values
avg_auc = {i: np.mean([roc_auc_list[j][i] for j in range(5)]) for i in range(n_classes)}
print("Average AUC Values:")
print(avg_auc)

# Performance Metrics

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def calculate_metrics(y_true, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, average='weighted'),
        'Recall': recall_score(y_true, y_pred, average='weighted'),
        'F1-score': f1_score(y_true, y_pred, average='weighted')
    }

# Initialize lists to store metrics for each fold
fold_metrics = []

# Calculate metrics for each fold
for fold, (train_index, test_index) in enumerate(kf.split(X_augmented), 1):
    X_test_fold = X_augmented[test_index]
    y_test_fold = y_augmented[test_index]
    
    # Get the model predictions (assuming you've saved the models from each fold)
    y_pred = model.predict(X_test_fold)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test_fold, axis=1)
    
    # Calculate metrics
    metrics = calculate_metrics(y_true_classes, y_pred_classes)
    metrics['Fold'] = fold
    fold_metrics.append(metrics)

# Create a DataFrame with the results
results_df = pd.DataFrame(fold_metrics)

# Calculate mean and standard deviation of metrics across folds
mean_metrics = results_df.mean().round(3)
std_metrics = results_df.std().round(3)

# Add mean and std to the results DataFrame
results_df = pd.concat([results_df, 
                        pd.DataFrame([mean_metrics], index=['Mean']),
                        pd.DataFrame([std_metrics], index=['Std'])
                       ]).reset_index()

# Rename the 'index' column to 'Fold' for the Mean and Std rows
results_df.loc[results_df['index'].isin(['Mean', 'Std']), 'Fold'] = results_df['index']
results_df = results_df.drop('index', axis=1)

# Reorder columns to put 'Fold' first
cols = ['Fold'] + [col for col in results_df.columns if col != 'Fold']
results_df = results_df[cols]

# Display the results
print("Performance Metrics for Each Fold:")
print(results_df.to_string(index=False))

# Calculate and display overall metrics
print("\nOverall Performance:")
overall_metrics = calculate_metrics(
    np.argmax(y_augmented, axis=1),
    np.argmax(model.predict(X_augmented), axis=1)
)
for metric, value in overall_metrics.items():
    print(f"{metric}: {value:.3f}")

# Graphs and Plots

In [ ]:
# Matplotlib CM

import matplotlib.pyplot as plt
import seaborn as sns

# Assuming you have 3 classes
class_names = ['AD', 'MCI', 'HC']  # Replace with your actual class names

# Set up the matplotlib figure
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Confusion Matrices for Each Fold', fontsize=16)

# Flatten the axes array for easier indexing
axes = axes.flatten()

# Plot confusion matrix for each fold
for i, cm in enumerate(confusion_matrices):
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, ax=axes[i])
    axes[i].set_title(f'Fold {i+1}')
    axes[i].set_ylabel('True label')
    axes[i].set_xlabel('Predicted label')

# Plot average confusion matrix
sns.heatmap(avg_cm, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names, ax=axes[-1])
axes[-1].set_title('Average')
axes[-1].set_ylabel('True label')
axes[-1].set_xlabel('Predicted label')

# Remove the last (empty) subplot
fig.delaxes(axes[5])

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()